<a href="https://colab.research.google.com/github/RudreeshJoshi/Garbha_Kalyani/blob/main/Garbh_Kalyani.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install all the required libraries

!pip install -U -q pdfplumber tiktoken openai chromaDB sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 631.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Import all the required Libraries

import pdfplumber
from pathlib import Path
import pandas as pd
from operator import itemgetter
import json
import tiktoken
import openai
import chromadb

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
single_pdf_path = '/content/drive/MyDrive/Gen_AI/GK_Doc/GK_Doc1.pdf'

In [5]:
# Open the PDF file
with pdfplumber.open(single_pdf_path) as pdf:

    # Get one of the pages from the PDF and examine it
    single_page = pdf.pages[0]

    # Extract text from the first page
    text = single_page.extract_text()

    # Extract tables from the first page
    tables = single_page.extract_tables()

    # Print the extracted text
    print(text)

What is Garbh Sanskar?
Garbh Sanskar is an ancient Indian science that emphasizes a positive, joyful,
stress-free, and virtuous lifestyle for the pregnant mother. It is the mental and emotional
diet for an unborn child.
If the fruits are to be good, the seeds must be sown well, and the plants must be well
cared for. Once the tree grows, there is no improvement in its fruit. Human life works on
this principle. If you want to get a child with the best qualities and instill the virtues in the
child, it must start from the mother's womb.
From which month should one start practicing Garbhsanskar? or when
to do garbh sanskar? or when to start garbh sanskar?
"Garbh Sanskar, from which month," or "from when to start Garbh sanskar?"
is the most common question pregnant mothers ask. Garbhsanskar is the lifestyle that
has to be practiced daily. One can start practicing Garbhsanskar from any day or month
of pregnancy. Garbhsanskar begins with pregnancy planning and lasts till the last day of
pregn

In [6]:
tables[0]

[['What is Garbh Sanskar?'],
 ['Garbh Sanskar is an ancient Indian science that emphasizes a positive, joyful,'],
 ['stress-free, and virtuous lifestyle for the pregnant mother. It is the mental and emotional'],
 ['diet for an unborn child.'],
 ['If the fruits are to be good, the seeds must be sown well, and the plants must be well'],
 ['cared for. Once the tree grows, there is no improvement in its fruit. Human life works on'],
 ['this principle. If you want to get a child with the best qualities and instill the virtues in the'],
 ["child, it must start from the mother's womb."],
 ['From which month should one start practicing Garbhsanskar? or when'],
 ['to do garbh sanskar? or when to start garbh sanskar?'],
 ['"Garbh Sanskar, from which month," or "from when to start Garbh sanskar?"'],
 ['is the most common question pregnant mothers ask. Garbhsanskar is the lifestyle that'],
 ['has to be practiced daily. One can start practicing Garbhsanskar from any day or month'],
 ['of pregnancy.

In [7]:
pdf_path = "/content/drive/MyDrive/Gen_AI/GK_Doc"

In [8]:
def check_bboxes(word, table_bbox):
    # Check whether word is inside a table bbox.
    l = word['x0'], word['top'], word['x1'], word['bottom']
    r = table_bbox
    return l[0] > r[0] and l[1] > r[1] and l[2] < r[2] and l[3] < r[3]

In [9]:
def extract_text_from_pdf(pdf_path):
    p = 0
    full_text = []


    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_no = f"Page {p+1}"
            text = page.extract_text()

            tables = page.find_tables()
            table_bboxes = [i.bbox for i in tables]
            tables = [{'table': i.extract(), 'top': i.bbox[1]} for i in tables]
            non_table_words = [word for word in page.extract_words() if not any(
                [check_bboxes(word, table_bbox) for table_bbox in table_bboxes])]
            lines = []

            for cluster in pdfplumber.utils.cluster_objects(non_table_words + tables, itemgetter('top'), tolerance=5):

                if 'text' in cluster[0]:
                    try:
                        lines.append(' '.join([i['text'] for i in cluster]))
                    except KeyError:
                        pass

                elif 'table' in cluster[0]:
                    lines.append(json.dumps(cluster[0]['table']))


            full_text.append([page_no, " ".join(lines)])
            p +=1

    return full_text

In [10]:
# Define the directory containing the PDF files
pdf_directory = Path(pdf_path)

# Initialize an empty list to store the extracted texts and document names
data = []

# Loop through all files in the directory
for pdf_path in pdf_directory.glob("*.pdf"):

    # Process the PDF file
    print(f"...Processing {pdf_path.name}")

    # Call the function to extract the text from the PDF
    extracted_text = extract_text_from_pdf(pdf_path)

    # Convert the extracted list to a PDF, and add a column to store document names
    extracted_text_df = pd.DataFrame(extracted_text, columns=['Page No.', 'Page_Text'])
    extracted_text_df['Document Name'] = pdf_path.name

    # Append the extracted text and document name to the list
    data.append(extracted_text_df)

    # Print a message to indicate progress
    print(f"Finished processing {pdf_path.name}")

# Print a message to indicate all PDFs have been processed
print("All PDFs have been processed.")

...Processing GK_Doc1.pdf
Finished processing GK_Doc1.pdf
...Processing GK_Doc2.pdf
Finished processing GK_Doc2.pdf
...Processing GK_Doc3.pdf
Finished processing GK_Doc3.pdf
All PDFs have been processed.


In [11]:
gk_pdfs_data = pd.concat(data, ignore_index=True)

In [12]:
gk_pdfs_data

,Page No.,Page_Text,Document Name
0,Page 1,"[[""What is Garbh Sanskar?""], [""Garbh Sanskar i...",GK_Doc1.pdf
1,Page 2,The garbhsanskar parent's you What Garbhsanska...,GK_Doc1.pdf
2,Page 3,if positive How baby Some the Stimulate help o...,GK_Doc1.pdf
3,Page 4,"[[""How to give garbh sanskar in the womb? or H...",GK_Doc1.pdf
4,Page 5,Sanskar provides them practicing I Will Online...,GK_Doc1.pdf
5,Page 6,Sanskar provides keep Garbh Can't It to Howeve...,GK_Doc1.pdf
6,Page 7,health. I could Garbh throughout while The pro...,GK_Doc1.pdf
7,Page 8,working effective listen or Should It's import...,GK_Doc1.pdf
8,Page 9,highly and During important It's trimester las...,GK_Doc1.pdf
9,Page 10,of specifically reduces Here 1. Classical Indi...,GK_Doc1.pdf


In [13]:
gk_pdfs_data.Page_Text[2]

'if positive How baby Some the Stimulate help or brain development Practice meditation development. Remember influenced for Garbhsansakr and development. provide and'

In [14]:
gk_pdfs_data['Text_Length'] = gk_pdfs_data['Page_Text'].apply(lambda x: len(x.split(' ')))

In [15]:
gk_pdfs_data['Text_Length']

,Text_Length
0,283
1,23
2,22
3,281
4,23
5,23
6,22
7,23
8,22
9,204


In [16]:
gk_pdfs_data = gk_pdfs_data.loc[gk_pdfs_data['Text_Length'] >= 10]
gk_pdfs_data

,Page No.,Page_Text,Document Name,Text_Length
0,Page 1,"[[""What is Garbh Sanskar?""], [""Garbh Sanskar i...",GK_Doc1.pdf,283
1,Page 2,The garbhsanskar parent's you What Garbhsanska...,GK_Doc1.pdf,23
2,Page 3,if positive How baby Some the Stimulate help o...,GK_Doc1.pdf,22
3,Page 4,"[[""How to give garbh sanskar in the womb? or H...",GK_Doc1.pdf,281
4,Page 5,Sanskar provides them practicing I Will Online...,GK_Doc1.pdf,23
5,Page 6,Sanskar provides keep Garbh Can't It to Howeve...,GK_Doc1.pdf,23
6,Page 7,health. I could Garbh throughout while The pro...,GK_Doc1.pdf,22
7,Page 8,working effective listen or Should It's import...,GK_Doc1.pdf,23
8,Page 9,highly and During important It's trimester las...,GK_Doc1.pdf,22
9,Page 10,of specifically reduces Here 1. Classical Indi...,GK_Doc1.pdf,204


In [17]:
gk_pdfs_data['Metadata'] = gk_pdfs_data.apply(lambda x: {'Document_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)

<ipython-input-17-6920a017b6f8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gk_pdfs_data['Metadata'] = gk_pdfs_data.apply(lambda x: {'Document_Name': x['Document Name'][:-4], 'Page_No.': x['Page No.']}, axis=1)


In [18]:
gk_pdfs_data['Metadata']

,Metadata
0,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
1,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
2,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
3,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
4,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
5,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
6,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
7,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
8,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
9,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."


In [19]:
filepath = "/content/drive/My Drive/Gen_AI/"

with open(filepath + "new_api_key.txt", "r") as f:
  openai.api_key = ' '.join(f.readlines())

In [20]:

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

In [21]:

chroma_data_path = '/content/drive/MyDrive/Gen_AI/GK_Doc'

In [22]:
import chromadb

In [23]:
client = chromadb.PersistentClient()

In [24]:

model = "text-embedding-ada-002"
embedding_function = OpenAIEmbeddingFunction(api_key=openai.api_key, model_name=model)

In [25]:

gk_collection = client.get_or_create_collection(name='RAG_on_Pregnancy', embedding_function=embedding_function)

In [26]:
# Convert the page text and metadata from your dataframe to lists to be able to pass it to chroma

documents_list = gk_pdfs_data["Page_Text"].tolist()
metadata_list = gk_pdfs_data['Metadata'].tolist()

In [27]:
gk_collection.add(
    documents= documents_list,
    ids = [str(i) for i in range(0, len(documents_list))],
    metadatas = metadata_list
)

In [28]:
gk_collection.get(
    ids = ['0','1','2'],
    include = ['embeddings', 'documents', 'metadatas']
)

{'ids': ['0', '1', '2'],
 'embeddings': [[0.010382060892879963,
   0.007737450301647186,
   0.012561901472508907,
   -0.02843611128628254,
   0.00130021246150136,
   -0.014558451250195503,
   -0.023709852248430252,
   -0.017870761454105377,
   -0.031552039086818695,
   -0.01421805564314127,
   -0.005639436189085245,
   0.00688646174967289,
   -0.020764121785759926,
   0.006536247208714485,
   -0.0004807267396245152,
   0.007233403157442808,
   0.048257600516080856,
   -0.01137706357985735,
   0.020044054836034775,
   0.00824804324656725,
   -0.01656154729425907,
   0.016352074220776558,
   -0.0010342784225940704,
   0.007030475419014692,
   -0.016064045950770378,
   0.012182229198515415,
   0.03642231225967407,
   -0.012660092674195766,
   -0.009668540209531784,
   0.0012167501263320446,
   -0.029483482241630554,
   -0.003525056643411517,
   -0.030923618003726006,
   -0.02008333057165146,
   -0.01750418171286583,
   -0.03477270528674126,
   -0.006025654263794422,
   -0.0060485652647912

In [29]:
cache_collection = client.get_or_create_collection(name='GK_Cache', embedding_function=embedding_function)

In [30]:
cache_collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None,
 'included': ['embeddings', 'metadatas', 'documents']}

In [31]:
query = input()

Nutrition&Diet


In [32]:
cache_results = cache_collection.query(
    query_texts=query,
    n_results=1
)

In [33]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = gk_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      #for key, val in results.items():
      #  if key != 'embeddings':
      #    for i in range(10):
      #      Keys.append(str(key)+str(i))
      #      Values.append(str(val[0][i]))
      for key, val in results.items():
        if key != 'embeddings':
          if val is not None and val[0] is not None:
            for i in range(10):
              Keys.append(str(key)+str(i))
              print(val[0][i])
              Values.append(str(val[0][i]))

      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })

25
24
23
6
28
1
21
0
12
4
0.45179591587552875
0.4715909348631883
0.47843096681362907
0.47893607282381306
0.4905307779915597
0.502267664098461
0.5050818174034288
0.5109280043149744
0.5123519612825398
0.5141693823079243
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 4'}
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 3'}
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 2'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 7'}
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 7'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 2'}
{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page 11'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 1'}
{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page 2'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 5'}
AssociatedtreatmentswhichenhancetheinternalenvironmentinthewomblikeUttarbasti, PichudharanandDhawan. Dietaryguidance. Yoga,exerciseandmeditation. Studyaboutprobabledaysforconception. (II) Pregnancy period (Garbh Sanskar) – stateemotionally,mentally,physicallyandspiritual

IndexError: string index out of range

In [39]:
# Implementing Cache in Semantic Search

# Set a threshold for cache search
threshold = 0.2

ids = []
documents = []
distances = []
metadatas = []
results_df = pd.DataFrame()


# If the distance is greater than the threshold, then return the results from the main collection.

if cache_results['distances'][0] == [] or cache_results['distances'][0][0] > threshold:
      # Query the collection against the user query and return the top 10 results
      results = gk_collection.query(
      query_texts=query,
      n_results=10
      )

      # Store the query in cache_collection as document w.r.t to ChromaDB so that it can be embedded and searched against later
      # Store retrieved text, ids, distances and metadatas in cache_collection as metadatas, so that they can be fetched easily if a query indeed matches to a query in cache
      Keys = []
      Values = []

      #for key, val in results.items():
      #  if key != 'embeddings':
      #    for i in range(10):
      #      Keys.append(str(key)+str(i))
      #      Values.append(str(val[0][i]))
      for key, val in results.items():
        if key != 'embeddings':
          if val is not None and val[0] is not None:
            # Check if val[0] is a list before iterating
            if isinstance(val[0], list):
              for i in range(10):
                Keys.append(str(key)+str(i))
                print(val[0][i])
                Values.append(str(val[0][i]))
            # If val[0] is not a list append it as a single value
            else:
                Keys.append(str(key))
                Values.append(str(val[0]))


      cache_collection.add(
          documents= [query],
          ids = [query],  # Or if you want to assign integers as IDs 0,1,2,.., then you can use "len(cache_results['documents'])" as will return the no. of queries currently in the cache and assign the next digit to the new query."
          metadatas = dict(zip(Keys, Values))
      )

      print("Not found in cache. Found in main collection.")

      result_dict = {'Metadatas': results['metadatas'][0], 'Documents': results['documents'][0], 'Distances': results['distances'][0], "IDs":results["ids"][0]}
      results_df = pd.DataFrame.from_dict(result_dict)
      results_df


# If the distance is, however, less than the threshold, you can return the results from cache

elif cache_results['distances'][0][0] <= threshold:
      cache_result_dict = cache_results['metadatas'][0][0]

      # Loop through each inner list and then through the dictionary
      for key, value in cache_result_dict.items():
          if 'ids' in key:
              ids.append(value)
          elif 'documents' in key:
              documents.append(value)
          elif 'distances' in key:
              distances.append(value)
          elif 'metadatas' in key:
              metadatas.append(value)

      print("Found in cache!")

      # Create a DataFrame
      results_df = pd.DataFrame({
        'IDs': ids,
        'Documents': documents,
        'Distances': distances,
        'Metadatas': metadatas
      })

25
24
23
6
28
1
21
0
12
4
0.45179591587552875
0.4715909348631883
0.47843096681362907
0.47893607282381306
0.4905307779915597
0.502267664098461
0.5050818174034288
0.5109280043149744
0.5123519612825398
0.5141693823079243
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 4'}
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 3'}
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 2'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 7'}
{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page 7'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 2'}
{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page 11'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 1'}
{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page 2'}
{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page 5'}
AssociatedtreatmentswhichenhancetheinternalenvironmentinthewomblikeUttarbasti, PichudharanandDhawan. Dietaryguidance. Yoga,exerciseandmeditation. Studyaboutprobabledaysforconception. (II) Pregnancy period (Garbh Sanskar) – stateemotionally,mentally,physicallyandspiritual

In [34]:
from sentence_transformers import CrossEncoder, util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [35]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [36]:
scores = cross_encoder.predict([['Nutrition & Diet', 'Spicy, too salty, fermented, highly flavored, pungent and stale foods should be avoided during pregnancy.'],['Is Nutrition a part of Antenatal Wellness??', 'There are some guidelines']])

In [37]:
scores

array([ -9.787899, -10.563093], dtype=float32)

In [40]:
cross_inputs = [[query, response] for response in results_df['Documents']]
cross_rerank_scores = cross_encoder.predict(cross_inputs)

In [41]:
cross_rerank_scores

array([ -6.4992895,  -7.804228 , -10.2087755, -11.09275  , -10.501323 ,
       -11.248575 , -11.056999 ,  -7.3150864,  -7.3366423, -11.242722 ],
      dtype=float32)

In [42]:

results_df['Reranked_scores'] = cross_rerank_scores

In [43]:
results_df

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...",Associatedtreatmentswhichenhancetheinternalenv...,0.451796,25,-6.499290
1,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...","thattheantenatalenvironmentiswellnourished,str...",0.471591,24,-7.804228
2,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...",opting sophisticatedformsothatitmakesyouproudb...,0.478431,23,-10.208776
3,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page...",health. I could Garbh throughout while The pro...,0.478936,6,-11.092750
4,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...",Colour therapy: believed yourbaby'seyes. Aroma...,0.490531,28,-10.501323
5,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page...",The garbhsanskar parent's you What Garbhsanska...,0.502268,1,-11.248575
6,"{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page...","Garbh Sanskar: Preparation of Motherhood [[""Na...",0.505082,21,-11.056999
7,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page...","[[""What is Garbh Sanskar?""], [""Garbh Sanskar i...",0.510928,0,-7.315086
8,"{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page...",Garbh Sanskar: Preparation of Motherhood There...,0.512352,12,-7.336642
9,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page...",Sanskar provides them practicing I Will Online...,0.514169,4,-11.242722


In [44]:
top_3_semantic = results_df.sort_values(by='Distances')
top_3_semantic[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...",Associatedtreatmentswhichenhancetheinternalenv...,0.451796,25,-6.499290
1,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...","thattheantenatalenvironmentiswellnourished,str...",0.471591,24,-7.804228
2,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...",opting sophisticatedformsothatitmakesyouproudb...,0.478431,23,-10.208776


In [45]:

top_3_rerank = results_df.sort_values(by='Reranked_scores', ascending=False)
top_3_rerank[:3]

,Metadatas,Documents,Distances,IDs,Reranked_scores
0,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page...",Associatedtreatmentswhichenhancetheinternalenv...,0.451796,25,-6.499290
7,"{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page...","[[""What is Garbh Sanskar?""], [""Garbh Sanskar i...",0.510928,0,-7.315086
8,"{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page...",Garbh Sanskar: Preparation of Motherhood There...,0.512352,12,-7.336642


In [46]:
top_3_RAG = top_3_rerank[["Documents", "Metadatas"]][:3]


In [47]:
top_3_RAG

,Documents,Metadatas
0,Associatedtreatmentswhichenhancetheinternalenv...,"{'Document_Name': 'GK_Doc3', 'Page_No.': 'Page..."
7,"[[""What is Garbh Sanskar?""], [""Garbh Sanskar i...","{'Document_Name': 'GK_Doc1', 'Page_No.': 'Page..."
8,Garbh Sanskar: Preparation of Motherhood There...,"{'Document_Name': 'GK_Doc2', 'Page_No.': 'Page..."


In [48]:
# Define the function to generate the response. Provide a comprehensive prompt that passes the user query and the top 3 results to the model

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the ayurvedic domain who can effectively answer user queries about  ayurvedic antenatal wellness."},
                {"role": "user", "content": f"""You are a helpful assistant in the ayurvedic domain who can effectively answer user queries about ayurvedic antenatal wellness.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of ayurvedic antenatal wellness documents in the dataframe '{top_3_RAG}'. These search results are essentially few page of an antenatal information document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the health document and the column 'metadata' contains the document name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant document names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the Document name(s) and page numbers(s) as citation. Refer to Metadat for getting page number
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.

                                                """},
              ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response['choices'][0]['message']['content'].split('\n')

In [50]:
!pip install --upgrade openai

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the ayurvedic domain who can effectively answer user queries about  ayurvedic antenatal wellness."},
                {"role": "user", "content": f"""You are a helpful assistant in the ayurvedic domain who can effectively answer user queries about ayurvedic antenatal wellness.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of ayurvedic antenatal wellness documents in the dataframe '{top_3_RAG}'. These search results are essentially few page of an antenatal information document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the health document and the column 'metadata' contains the document name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant document names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the Document name(s) and page numbers(s) as citation. Refer to Metadat for getting page number
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.

                                                """},
              ]

    # Use openai.ChatCompletion.create for versions of openai<1.0.0.
    #response = openai.ChatCompletion.create(
    #    model="gpt-3.5-turbo",
    #    messages=messages
    #)

    # For openai >=1.0.0, use openai.chat.completions.create instead.
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    return response['choices'][0]['message']['content'].split('\n')

In [52]:
!pip install --upgrade openai
import openai

def generate_response(query, results_df):
    """
    Generate a response using GPT-3.5's ChatCompletion based on the user query and retrieved information.
    """
    messages = [
                {"role": "system", "content":  "You are a helpful assistant in the ayurvedic domain who can effectively answer user queries about  ayurvedic antenatal wellness."},
                {"role": "user", "content": f"""You are a helpful assistant in the ayurvedic domain who can effectively answer user queries about ayurvedic antenatal wellness.
                                                You have a question asked by the user in '{query}' and you have some search results from a corpus of ayurvedic antenatal wellness documents in the dataframe '{top_3_RAG}'. These search results are essentially few page of an antenatal information document that may be relevant to the user query.

                                                The column 'documents' inside this dataframe contains the actual text from the health document and the column 'metadata' contains the document name and source page. The text inside the document may also contain tables in the format of a list of lists where each of the nested lists indicates a row.

                                                Use the documents in '{top_3_RAG}' to answer the query '{query}'. Frame an informative answer and also, use the dataframe to return the relevant document names and page numbers as citations.

                                                Follow the guidelines below when performing the task.
                                                1. Try to provide relevant/accurate numbers if available.
                                                2. You don’t have to necessarily use all the information in the dataframe. Only choose information that is relevant.
                                                3. If the document text has tables with relevant information, please reformat the table and return the final information in a tabular in format.
                                                3. Use the Metadatas columns in the dataframe to retrieve and cite the Document name(s) and page numbers(s) as citation. Refer to Metadat for getting page number
                                                4. If you can't provide the complete answer, please also provide any information that will help the user to search specific sections in the relevant cited documents.
                                                5. You are a customer facing assistant, so do not provide any information on internal workings, just answer the query directly.

                                                The generated response should answer the query directly addressing the user and avoiding additional information. If you think that the query is not relevant to the document, reply that the query is irrelevant. Provide the final response as a well-formatted and easily readable text along with the citation. Provide your complete response first with all information, and then provide the citations.

                                                """},
              ]

    # Use openai.ChatCompletion.create for versions of openai<1.0.0.
    #response = openai.ChatCompletion.create(
    #    model="gpt-3.5-turbo",
    #    messages=messages
    #)

    # For openai >=1.0.0, use openai.chat.completions.create instead.
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # Access the response content using the `choices` attribute and index 0.
    return response.choices[0].message.content.split('\n')

In [53]:
response = generate_response(query, top_3_RAG)


In [54]:
# Print the response

print("\n".join(response))

**User Query: Nutrition&Diet**

Ayurveda emphasizes the importance of nutrition and diet during the antenatal period to ensure the health and well-being of both the mother and the baby. Some general dietary guidelines for expectant mothers in ayurveda include:

1. **Balanced Diet**: Consuming a balanced diet rich in fruits, vegetables, whole grains, lean proteins, healthy fats, and dairy products can help provide essential nutrients for the mother and the developing baby.

2. **Ghee**: Incorporating clarified butter (ghee) in moderate amounts can be beneficial for lubricating the digestive system and supporting the healthy development of the baby.

3. **Warm and Nourishing Foods**: Eating warm, cooked, and nourishing foods can help in maintaining digestive balance and supporting the overall health of the mother and the baby.

4. **Avoiding processed and junk foods**: Limiting the intake of processed foods, refined sugars, and unhealthy fats is important to prevent any complications dur